In [ ]:
import pandas as pd
from pathlib import Path
from datetime import timedelta
import altair as alt
from IPython.display import Image
from altair_saver import save
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
data_path = Path("../Dataset/")

In [ ]:
training_data = pd.read_csv(data_path/"Candidate_tech_evaluation_candidate_copy_data science_fraud.csv").drop(columns=['Unnamed: 0'])

In [ ]:
training_data.head()

In [ ]:
ip_address = pd.read_excel(data_path/"Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx")

In [ ]:
ip_address.head()

In [ ]:
ip_address.shape

In [ ]:
ip_address['lower_bound_ip_address'].min(), ip_address['upper_bound_ip_address'].max()

In [ ]:
ip_address['upper_bound_ip_address'].max() > training_data['ip_address'].max()

In [ ]:
ip_address.loc[ip_address['country']=='United States', 'lower_bound_ip_address'].min()

In [ ]:
ip_address.loc[ip_address['country']=='United States', 'lower_bound_ip_address'].max()

In [ ]:
ip_address.loc[ip_address['country']=='Canada', 'lower_bound_ip_address'].min()

In [ ]:
training_data['ip_address'].min(), training_data['ip_address'].max()

In [ ]:
op1 = ip_address['lower_bound_ip_address'] >= training_data['ip_address'].min()
op2 = ip_address['upper_bound_ip_address'] <= training_data['ip_address'].max() 
ip_address = ip_address.loc[op1 & op2]

In [ ]:
ip_address.shape

In [ ]:
ip_address['country'].value_counts()

In [ ]:
from inter

In [ ]:
training_data['ip_address'].value_counts()

In [ ]:
def get_ip_country(ip):
    lower_bound = ip_address[ip_address['lower_bound_ip_address']<=ip]
    upper_bound = lower_bound[lower_bound['upper_bound_ip_address']>=ip]
    try:
        country = upper_bound['country'].to_numpy()[0]
    except:
        country = 'Not available'
    return country

In [ ]:
training_data_sample = training_data.sample(10)
training_data_sample['ip_country'] = training_data_sample['ip_address'].progress_apply(get_ip_country)

In [ ]:
training_data_sample.head()

In [ ]:
training_data['ip_country'] = training_data['ip_address'].progress_apply(get_ip_country)

In [ ]:
training_data.shape

In [ ]:
training_data.to_csv(data_path/"training_data_with_ipCountry.csv")

# Data size

In [ ]:
training_data.shape

The dataset is imblanced.  
About 9.7% of the record is fraud.  

In [ ]:
training_data['class'].value_counts()

In [ ]:
training_data['class'].value_counts()[0] / training_data['class'].value_counts()[1]

# Data time range

The transactions happend in 2015 and Any user did not signup after Aug 18th.  
Each time feature itself seems to be normal

In [ ]:
training_data['signup_time_dt'] = pd.to_datetime(training_data['signup_time'])
training_data['purchase_time_dt'] = pd.to_datetime(training_data['purchase_time'])

In [ ]:
training_data['signup_time_dt'].value_counts()

In [ ]:
training_data['signup_time_dt'].min(), training_data['signup_time_dt'].max()

In [ ]:
training_data['purchase_time_dt'].value_counts()

In [ ]:
training_data['purchase_time_dt'].min(), training_data['purchase_time_dt'].max()

## **Time Interval**
important feature

When counted each datetime value above, it seemed some records have same datetime for signup and purchase.  
The time difference range between signup and the first purchase is from 0 days to 119 days.  
0 days records are suspicious, especially the records that has no time interval

In [ ]:
training_data['days_signup_purchase'] = training_data['purchase_time_dt'] - training_data['signup_time_dt']

In [ ]:
training_data['days_signup_purchase'].value_counts().sort_index()

In [ ]:
training_data['days_signup_purchase'].min(), training_data['days_signup_purchase'].max()

convert timedelta into minutes

In [ ]:
training_data['minutes_signup_purchase'] = training_data['days_signup_purchase'].dt.total_seconds() / 60

In [ ]:
training_data['minutes_signup_purchase'].value_counts().sort_index()

In [ ]:
op1 = training_data['minutes_signup_purchase'] <= 20
time_delta_chart = alt.Chart(training_data.loc[op1,['minutes_signup_purchase', 'class']]).mark_bar().encode(
    alt.X('minutes_signup_purchase:N'),
    alt.Y('count()'),
    alt.Color('class:N')
).properties(
    title='Time delta'
)
time_delta_chart.save('./charts/time_delta_plot.png')
Image(filename='./charts/time_delta_plot.png')

In [ ]:
op1 = training_data['minutes_signup_purchase'] <= 1
training_data.loc[op1, 'class'].value_counts()

Among the records with a 0-day interval, the transactions that a purchase was made within 1 minutes are all fraud.  
It is about 53% of all fraud records. this time interval is significant feature for prediction indeed.  

In [ ]:
6021 / 11265

Records of fraud transactions are widely spread.

In [ ]:
op1 = training_data['class'] == 1
training_data.loc[op1, 'signup_time_dt'].min(), training_data.loc[op1, 'signup_time_dt'].max()

In [ ]:
op1 = training_data['class'] == 1
training_data.loc[op1, 'purchase_time_dt'].min(), training_data.loc[op1, 'purchase_time_dt'].max()

## Time
Let's check if the records shows fraud occurred during specific time period.  

In [ ]:
training_data['signup_time_h_dt'] = training_data['signup_time_dt'].dt.hour
training_data['purchase_time_h_dt'] = training_data['purchase_time_dt'].dt.hour

There is no specific time period that fraud occurred a lot.  

In [ ]:
op1 = training_data['class'] == 1
training_data.loc[op1, ['signup_time_h_dt']].value_counts().sort_index()

In [ ]:
op1 = training_data['class'] == 1
training_data.loc[op1, ['purchase_time_h_dt']].value_counts().sort_index()

# Purchase values

Histogram and box plot shows that it is not common for the purchase value to exceeds 80 dollars.

In [ ]:
training_data['purchase_value'].hist(figsize=(5,2))

To see if such records are related to fraud, I divided records by quantiles.  

In [ ]:
training_data[['purchase_value']].describe()

In [ ]:
training_data[['purchase_value']].boxplot(figsize=(4,4))

In [ ]:
def get_purchase_range(purchase):
    if purchase <= 22:
        return 'purchase_Q1'
    elif purchase in range(23, 34):
        return 'purchase_Q2'
    elif purchase in range(34, 49):
        return 'purchase_Q3'
    else:
        return 'purchase_upper_Q'
    

In [ ]:
training_data['purchase_range'] = training_data['purchase_value'].apply(get_purchase_range)

As we can see in pivot table, each quantile area shows about 9 to 10% of fraud records proportion which is not 

In [ ]:
purchase_pivot = pd.pivot_table(training_data, index='purchase_range', values=['user_id'], columns=['class'], aggfunc='count')
purchase_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
purchase_pivot['proportion'] = round(purchase_pivot['1'] / purchase_pivot.sum(axis=1), 2)
purchase_pivot

# **DEVICE**
import feature

Some device is used for multiple users.  
In most cases, one device was used less than 3 times.  
Therefore, if same device appear more than 3 times, it is suspicious.

In [ ]:
training_data['device_id'].value_counts().hist(figsize=(5,2))

Counted how many times the device appears in dataset and map the appearance number as 'device_count' column.

In [ ]:
device_map = training_data['device_id'].value_counts().to_dict()

In [ ]:
training_data['device_count'] = training_data['device_id'].map(device_map)

In [ ]:
op1 = training_data['device_count'] >= 4
training_data.loc[op1, 'class'].value_counts()

The transaction records that are made by devices used more than 3 times are highly fraud records.  
The proportion of fraud records in each devices is more than 80%.

In [ ]:
device_count_chart = alt.Chart(training_data[['device_count','source', 'class']]).mark_bar().encode(
    alt.X('device_count'),
    alt.Y('count()').sort('-x').stack("normalize"),
    alt.Color('class:N')
).properties(
    title='Device count'
)
device_count_chart.save('./charts/device_count_plot.png')
Image(filename='./charts/device_count_plot.png')

# Source

There is no significant difference between source groups.

In [ ]:
source_pivot = pd.pivot_table(training_data, index='source', values=['user_id'], columns=['class'], aggfunc='count').loc[['SEO', 'Ads', 'Direct']]
source_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
source_pivot['proportion'] = round(source_pivot['1'] / source_pivot.sum(axis=1), 2)
source_pivot

In [ ]:
source_chart = alt.Chart(training_data[['purchase_value','source', 'class']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('source:N').sort('-x'),
    alt.Color('class:N')
)
source_chart.save('./charts/source_chart.png')
Image(filename='./charts/source_chart.png')

I checked if there are some difference by time for each source group.  
But I could not find meaningful difference by time.  
So I made time range as below.  
- 6 to 11 : morning
- 12 to 17 : afternoon
- 18 to 23 : evening
- 24 to 5 : night

In [ ]:
def set_time_group(time):
    if time in range(6, 12):
        return 'morning'
    elif time in range(12, 18):
        return 'afternoon'
    elif time in range(18, 24):
        return 'evening'
    else:
        return 'night'

In [ ]:
training_data['sign_time_range'] = training_data['signup_time_h_dt'].apply(set_time_group)
training_data['purchase_time_range'] = training_data['purchase_time_h_dt'].apply(set_time_group)

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('purchase_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='SEO')
            )

signup_chart = alt.Chart(training_data[['source', 'signup_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('signup_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='SEO')
            )

seo_time_chart = purchase_chart | signup_chart
seo_time_chart.save('./charts/seo_time_chart.png')
Image(filename='./charts/seo_time_chart.png')

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_range', 'class']]).mark_bar().encode(
                alt.X('purchase_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='SEO')
            )

signup_chart = alt.Chart(training_data[['source', 'sign_time_range', 'class']]).mark_bar().encode(
                alt.X('sign_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='SEO')
            )

seo_time_range_chart = purchase_chart | signup_chart
seo_time_range_chart.save('./charts/seo_time_range_chart.png')
Image('./charts/seo_time_range_chart.png')

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('purchase_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Ads')
            )

signup_chart = alt.Chart(training_data[['source', 'signup_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('signup_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Ads')
            )

ads_time_chart = purchase_chart | signup_chart
ads_time_chart.save('./charts/ads_time_chart.png')
Image('./charts/ads_time_chart.png')

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_range', 'class']]).mark_bar().encode(
                alt.X('purchase_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Ads')
            )

signup_chart = alt.Chart(training_data[['source', 'sign_time_range', 'class']]).mark_bar().encode(
                alt.X('sign_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Ads')
            )

ads_time_range_chart = purchase_chart | signup_chart
ads_time_range_chart.save('./charts/ads_time_range_chart.png')
Image('./charts/ads_time_range_chart.png')

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('purchase_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Direct')
            )

signup_chart = alt.Chart(training_data[['source', 'signup_time_h_dt', 'class']]).mark_bar().encode(
                alt.X('signup_time_h_dt'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Direct')
            )

direct_time_chart = purchase_chart | signup_chart
direct_time_chart.save('./charts/direct_time_chart.png')
Image('./charts/direct_time_chart.png')

In [ ]:
purchase_chart = alt.Chart(training_data[['source', 'purchase_time_range', 'class']]).mark_bar().encode(
                alt.X('purchase_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Direct')
            )

signup_chart = alt.Chart(training_data[['source', 'sign_time_range', 'class']]).mark_bar().encode(
                alt.X('sign_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='source', equal='Direct')
            )

direct_time_range_chart = purchase_chart | signup_chart
direct_time_range_chart.save('./charts/direct_time_range_chart.png')
Image('./charts/direct_time_range_chart.png')

# Browser

In [ ]:
browser_pivot = pd.pivot_table(training_data, index='browser', values=['user_id'], columns=['class'], aggfunc='count').loc[['Chrome','IE', 'Safari','FireFox','Opera']]
browser_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
browser_pivot['proportion'] = round(browser_pivot['1'] / browser_pivot.sum(axis=1), 2)
browser_pivot

In [ ]:
browser_chart = alt.Chart(training_data[['purchase_value', 'browser', 'class']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('browser:N').sort('-x'),
    alt.Color('class:N')
)
browser_chart.save('./charts/browser_chart.png')
Image('./charts/browser_chart.png')

# SEX

In [ ]:
sex_pivot = pd.pivot_table(training_data, index='sex', values=['user_id'], columns=['class'], aggfunc='count').loc[['M','F']]
sex_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
sex_pivot['proportion'] = round(sex_pivot['1'] / sex_pivot.sum(axis=1), 2)
sex_pivot

In [ ]:
sex_chart = alt.Chart(training_data[['purchase_value','sex', 'class']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('sex:N').sort('-x'),
    alt.Color('class:N')
)
sex_chart.save('./charts/sex_chart.png')
Image('./charts/sex_chart.png')

# AGE

When look into proportion of fraud transaction by age group, some age group shows higher rate than others.  
However, It is not meaningful to separate transaction by each age.  
For example, 63 year old transaction shows high proportion, but 62 or 64 has low proportion.  
It is hard to think that 63 year old and 62 year old is that different.  
And transaction record of 63 year old is not that many.  
It would be better to set some age range.

In [ ]:
age_pivot = pd.pivot_table(training_data, index='age', values=['user_id'], columns=['class'], aggfunc='count')
age_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
age_pivot['proportion'] = round(age_pivot['1'] / age_pivot.sum(axis=1), 2)
age_pivot.sort_values(by='proportion')

In [ ]:
age_chart = alt.Chart(training_data[['purchase_value','age', 'class']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('age:N', axis=alt.Axis(grid=True)),
    alt.Color('class:N')
)
age_chart.save('./charts/age_chart.png')
Image('./charts/age_chart.png')

I divided the transactions into several age groups: 10 years interval.  
It appears that in age group, there is no significant difference.  

In [ ]:
def get_age_group(age):
    if age in range(18, 28):
        return 'age 1'
    elif age in range(28, 38):
        return 'age 2'
    elif age in range(38, 48):
        return 'age 3'
    elif age in range(48, 58):
        return 'age 4'
    elif age in range(58, 67):
        return 'age 5'
    else:
        return 'age 6'

In [ ]:
training_data['age_group'] = training_data['age'].apply(get_age_group)

In [ ]:
age_group_pivot = pd.pivot_table(training_data, index='age_group', values=['user_id'], columns=['class'], aggfunc='count')
age_group_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
age_group_pivot['proportion'] = round(age_group_pivot['1'] / age_group_pivot.sum(axis=1), 2)
age_group_pivot

In [ ]:
age_group_chart = alt.Chart(training_data[['purchase_value','age_group', 'class']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('age_group:N', axis=alt.Axis(grid=True)),
    alt.Color('class:N')
)
age_group_chart.save('./charts/age_group_chart.png')
Image('./charts/age_group_chart.png')

In [ ]:
def get_age_group2(age):
    if age in range(18, 23):
        return 'age 01'
    elif age in range(23, 28):
        return 'age 02'
    elif age in range(28, 33):
        return 'age 03'
    elif age in range(33, 38):
        return 'age 04'
    elif age in range(38, 43):
        return 'age 05'
    elif age in range(43, 49):
        return 'age 06'
    elif age in range(49, 54):
        return 'age 07'
    elif age in range(54, 59):
        return 'age 08'
    elif age in range(59, 64):
        return 'age 09'
    elif age in range(64, 69):
        return 'age 10'
    else:
        return 'age 11'

In [ ]:
training_data['age_group2'] = training_data['age'].apply(get_age_group2)

In [ ]:
age_group_pivot2 = pd.pivot_table(training_data, index='age_group2', values=['user_id'], columns=['class'], aggfunc='count')
age_group_pivot2.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
age_group_pivot2['proportion'] = round(age_group_pivot2['1'] / age_group_pivot2.sum(axis=1), 2)
age_group_pivot2.sort_index()

# **Country**
importand feature  
proportion of fraud records shows difference by country group.

In [ ]:
17418 / 120000

Most of the transaction occurred at United States.  
And about 14.5% of the tranasaction is not determined the country by ip bound.

In [ ]:
training_data['ip_country'].value_counts()

In [ ]:
country_proportion = training_data[training_data['class']==1].groupby('ip_country').size() / training_data.groupby('ip_country').size()
country_proportion = country_proportion.fillna(0).reset_index(name='proportion_country')

In [ ]:
country_proportion.sort_values(by='proportion_country', ascending=False)

In [ ]:
sorted_data = pd.merge(training_data, country_proportion, on='ip_country', how='left')

In [ ]:
sorted_data.head()

In [ ]:
age_chart = alt.Chart(sorted_data[['ip_country', 'class', 'proportion_country']]).mark_bar().encode(
    alt.X('count()').stack('normalize'),
    alt.Y('ip_country:N', axis=alt.Axis(grid=True), sort=alt.EncodingSortField(field='proportion_country', order='descending')),
    alt.Color('class:N')
)
age_chart.save('./charts/ip_country.png')
Image('./charts/ip_country.png')

### Country & purchasement value

Fraud Purchase value distribution for each country:  
If we plot mean of purchase value of fraud and non-fraud, there might be difference between countries.  
Interestingly, some country shows mean of fraud purchase amount is lower than mean of non fraud purchasement amount.  
In either way, there is some difference.  

In [ ]:
training_data.columns

In [ ]:
pd.pivot_table(training_data, index=['ip_country'], values=['purchase_value'], columns=['class'], aggfunc='mean').sort_values(by=('purchase_value',1), ascending=False)

In [ ]:
age_chart = alt.Chart(sorted_data[['ip_country', 'class', 'purchase_value']]).mark_bar().encode(
    alt.X('mean(purchase_value):Q'),
    alt.Y('ip_country:N'),
    alt.Color('class:N')
)
age_chart.save('./charts/country_fraud_purchase_amount.png')
Image('./charts/country_fraud_purchase_amount.png')

### Country & Source:
Source data can be useful when it combines with country data.  
For example, New Zealand shows the proportion of fraud records as 21%.  
But if source of the transaction is Direct, proportion decrease to 5%.  

In [ ]:
country_source_pivot = pd.pivot_table(training_data, index=['ip_country','source'], values=['user_id'], columns=['class'], aggfunc='count')
country_source_pivot = country_source_pivot.fillna(0)
country_source_pivot.columns = ['0','1']
# print(purchase_pivot.sum(axis=1))
country_source_pivot['proportion'] = round(country_source_pivot['1'] / country_source_pivot.sum(axis=1), 2)
country_source_pivot

### Country & time range
When look into some country that has high proportion of fraud transaction records,  
the proportion of fraud transaction is different by time.  

In [ ]:
purchase_chart = alt.Chart(training_data[['ip_country', 'purchase_time_range', 'class']]).mark_bar().encode(
                alt.X('purchase_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='ip_country', equal='New Zealand')
            )

signup_chart = alt.Chart(training_data[['ip_country', 'sign_time_range', 'class']]).mark_bar().encode(
                alt.X('sign_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='ip_country', equal='New Zealand')
            )

direct_time_range_chart = purchase_chart | signup_chart
direct_time_range_chart.save('./charts/Newzealand_time_range_chart.png')
Image('./charts/Newzealand_time_range_chart.png')

But in case the country has low fraud traction proportion, it seems time is nothing to do with fraud transaction.  

In [ ]:
purchase_chart = alt.Chart(training_data[['ip_country', 'purchase_time_range', 'class']]).mark_bar().encode(
                alt.X('purchase_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='ip_country', equal='United States')
            )

signup_chart = alt.Chart(training_data[['ip_country', 'sign_time_range', 'class']]).mark_bar().encode(
                alt.X('sign_time_range'),
                alt.Y('count()').stack('normalize'),
                alt.Color('class:N')
            ).transform_filter(
                alt.FieldEqualPredicate(field='ip_country', equal='United States')
            )

direct_time_range_chart = purchase_chart | signup_chart
direct_time_range_chart.save('./charts/UnitedStates_time_range_chart.png')
Image('./charts/UnitedStates_time_range_chart.png')

# Compare value of False positive and False negative
False negative: If model predict fraud as normal  
False positive: If model predict normal as fraud

In [ ]:
op1 = training_data['class'] == 1
training_data.loc[op1, ['purchase_value']].mean()

If False negative occurr Electronica will lose approximatelly \\$37 per record.  
On the other hand, If False positive occurr Electronica will lose \\$8 per record.  
Therefore model should prevent False negative more than False positive.  
=> Recall score should be high.  

# Conclusion

Through the EDA, I concluded that below features would be helpful to predict fraud records.
- Time interval between signup and the first purchasement in minutes.
- Device_id frequancy: the number that the device id appeared in dataset.
- Country: determined by numeric ip address boundary.

Important metric to evaluate the model: Recall score